## The Question
Large Language Models perform really well on natural language generation tasks, sometimes equally or surpassing human abilities. Since GPT, natural language processing has seen many advancements, and ChatGPT showcased the power of LLMs on variety of tasks like question answering, generating long texts, and even simple logical reasoning. But, the answers were based out of the parametric memory of the model which was frozen, i.e., it could not update or fetch relevant data. Many a times the LLMs would invent results and output them: they would *hallucinate*. **Is there a way to make large language models fetch relevant information and generate accurate answers?**

## The Dataset
We are utilizing 3 chapters (chapter 1, 2, and 7) of the course textbook 'Artificial Intelligence: A Modern Approach' and a couple of slides from Prof. Keogh and Prof. LePendu as our additional context.

## The Method
To achieve our objective of eliminating hallucination and boost relevancy, we utilize Llama-Index, LangChain, and Ollama to build a RAG pipeline. Llama-Index is used to index the documents and store them in a vector database. LangChain serves the local Ollama model of Llama 2 7B. The chapters from the textbook and the slides are index and stored in a vector database which are then retrieved based on the similarity with the query passed to the language model. We have used OpenAI's state-of-the-art text-embedding-ada-002 embedding model through their API. 

## Our hypothesis
We feel that supplementing the LLM with additional context can boost the relevance of the answer and stop it from hallucinating. The evaluation is done using the RAGAS approach where we measure how relevant and consistent the generated result is to the context and the query.   

## Why bother?
As the use of LLM is rising, one LLM cannot know everything in this universe. It is important to have specific LLM for a specific purpose. It is also important to have fresh data for the LLM to access and output relevant and factually correct results. The approach of RAG seems promising to alleviate most of the problems related to LLMs generation and correct usage can boost the utilization of the language models in various fields. 






# Main Jupyter Notebook for the CS205 Final Project
*This only works if run locally after completing all the installations mentioned in the README file of the project*

1. In this project we will explore a usecase of Large Language Models
2. We will start with how to use an LLM through HuggingFace, and explain some of the basic concepts behind an LLM. 
3. Once we have a good understading of how to use an LLM for generating text, we will explore Retrieval Augmented Generation (RAG). 

 For this project we have used Llama 2 7 Billion paramter model with OpenAI's text-embed-002 embedding model. Llama 2 7B was served locally by Ollama. We have used Llama Index and LangChain to interact with the LLM

#### The data store is at the root of the project directory with the name 'data'. Create a data repository before running the indexing and query cells

#### Let's understand how to use an LLM using HuggingFace

In [77]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import os

In [ ]:
import torch

#Import HuggingFace Transformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#Fetch Meta's OPT LLM with 1.3 billion parameters. This is quite a small model compared to the SOTA like GPT4V, etc.

model = AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b')
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-1.3b')

##### Open Pre-trained Transformer (OPT) is a collection of decoder-only transformer developed by Meta. 

In [ ]:
input_text = 'I like CS205 Artificial Intelligence course, because'
tok_input = tokenizer(input_text, return_tensors='pt', add_special_tokens=True, truncation=True) #Create tokens from the given input and return PyTorch tensors

In [ ]:
torch.manual_seed(123)

generated_output = model.generate(**tok_input, 
                                  max_new_tokens=200, 
                                  return_dict_in_generate=True, 
                                  do_sample=True) #Generation is deterministic. 
                                                  #To use top-k sampling, set do_sample=True to get different responses in each generation
                                                  #Set do_sample=False to have a deterministic generation each time


In [ ]:
decoded_output = tokenizer.batch_decode(generated_output.sequences, skip_special_tokens=True)[0]
print(decoded_output)

#### We saw text generation in the previous subsection. Now lets explore text summarization, a critical usecase of LLM

#### What is an embedding?

Embedding is a numerical representation of the text. The words in the vocabulary are mapped to a set of integers. These integers are then converted into another mathematical representation. This 'embedding' vector is of shape (n_tokens x embedding_dimension)

In [ ]:
sentence = "Life is sweet and sugary, but eating ice cream on Everest is long"

In [ ]:
tokenize1 = {s: i for i, s in enumerate(sorted(sentence.replace(',', '').split()))}

In [ ]:
sentence_vec = torch.tensor([tokenize1[s] for s in sentence.replace(',', '').split()])
print(sentence_vec)

The sentence1 is tokenized as shown in the output above. The words are mapped to an integer. 

In [ ]:
torch.manual_seed(123)

In [ ]:
embed = torch.nn.Embedding(len(sentence_vec), 10)
embedded_sentence = embed(sentence_vec).detach()

The sentence "Life is sweet and sugary, but eating ice cream in Manali is long" now has a representation like shown below. It is converted from text to a form which the machine understands. This embedding is the input to a language model.

In [ ]:
print(embedded_sentence.shape)
print(embedded_sentence)

In [ ]:
sentence1 = "I feel like driving car today"
sentence2 = "I think I want to drive today"

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
embedding1 = model.encode(sentence1)
embedding2 = model.encode(sentence2)

Evaluating a cosine similarity of 2 embeddings tells us how similar the vectors are semantically. This is particularly useful in our application where the query is matched with documents using a similarity function. 

In [ ]:
util.pytorch_cos_sim(embedding1, embedding2)

#### What is RAG?

#### Retrieval Augmented Generation 

It is a technique in generative AI to boost the knowledge of an LLM. The LLM parameters are learned and not updated to the current information, so a specialized database of knowledge (could be private) is created for the LLM is access. This is a non-parametric memory, i.e, this information is not stored in the learned paramaters of the LLM. RAG combines retrieval with generation for content generation tasks.

How does an RAG work?

1. Retrieval: The model retrieves a set of top-k relevant documents that act as additional context for the query. Since the documents are stored in the database in the form of embeddings, the model can perform similarity searches in retrieval

2. Generation: Once the documents are retrieved, this serves as additional context along with the original input. The generative model can now use both the original input and retrieved context to generated the content. 

#### Implementing RAG with Llama Index using ChromaDB as the vector database. Ollama is used to serve Llama 2 locally

The data can be accessed at this link. Add this folder to the root of the project directory

https://drive.google.com/drive/folders/10wzRErO4Zlj6L3bLSqh9QtTLDkaUOuxS?usp=drive_link

In [ ]:
import openai

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, download_loader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import OpenAIEmbedding

from langchain.llms import Ollama

import chromadb

In [ ]:
from dotenv import dotenv_values
from pathlib import Path

In [80]:
api_key = dotenv_values('../.env')["OPENAI_API_KEY"]
openai.api_key = api_key

os.environ["OPENAI_API_KEY"] = api_key # For RAGAS evaluation

In [ ]:
#Set the embedding

llm = Ollama(model="llama2")
#embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2") #Local Llama 2 embedding model
embed_model = OpenAIEmbedding() #Using OpenAI's text-embed-002

text-embedding-ada-002 is a powerful embedding model released by OpenAI. Like we discussed, an embedding represents a text mathematically in n-dimensions, and the distance between the embeddings can measure the similarity between the sentences or words. 

In [ ]:
COLLECTION = "aiprof"
SLIDE_COLLECTION = 'slides'
PATH = '../chroma'

In [ ]:
# create client and a new collection
db = chromadb.PersistentClient(path=PATH)
chroma_collection = db.get_or_create_collection(COLLECTION)

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [85]:
# load documents
documents = SimpleDirectoryReader("../data/AIMA/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path=PATH)
chroma_collection = db2.get_or_create_collection(COLLECTION)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index2 = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
query_engine = index2.as_query_engine()

In [ ]:
resp = query_engine.query("What is the turing test?")
print(resp.response)

In [ ]:
resp = query_engine.query("Generate 2 concise questions about rational agents")

In [ ]:
resp.response.split('\n')

#### Querying Slides (PPTx)

In [ ]:
# create client and a new collection
slides_db = chromadb.PersistentClient(path=PATH)
slides_chroma_collection = slides_db.get_or_create_collection(SLIDE_COLLECTION)

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=slides_chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [ ]:
slides_reader = download_loader("PptxReader")
loader = slides_reader()

print(type(SimpleDirectoryReader))

slides = loader.load_data(Path('../data/slides/4_Adversarial_Search.pptx'))

In [ ]:
slides_index = VectorStoreIndex.from_documents(documents=slides, 
                                               storage_context=storage_context, 
                                               service_context=service_context)

In [ ]:
slides_query = slides_index.as_query_engine()
resp = slides_query.query("What is The Minimax Algorithm?")
print(resp.response)

## Evaluation

We have been quite successful at getting what looks like accurate results, but lets run some evaluations on the generated text. Inspired from the RAGAS paper (https://arxiv.org/pdf/2309.15217v1.pdf), there is a ragas python library which runs evaluations on the RAG generated text. 

**Retrieval Augmentation Generated Assessment** evaluates a RAG generated text without human interference. Even after utilizing RAG we wouldn't be sure if the system hallucinated the generation. The paper proposes a suite of metrics that evaluates RAG.

*Faithfulness*: Measures the factual consistency of the generated answer. It is based on the answer and the retrieved context. The generated answer is faithful if all the answers can be deduced from the given context. The score ranges from 0 to 1. Higher the score, the better. 

*Answer Relevance*: Measures how relevant the answer is to the given prompt. Incomplete and/or redudant answers are given a lower score. The score ranges from 0 to 1.   

*Context Relevance*: Similar to answer relevancy, context relevancy measures the relevance of the generated text to the context. The idea is that the context should exlusively contain the information required to answer the query prompt. 

*Context Precision*: Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.

*Context Recall*: Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

Definitions are sourced from https://github.com/explodinggradients/ragas/tree/main/docs/concepts/metrics and the RAGAS paper. 

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [ ]:
from ragas.llama_index import evaluate

In [ ]:
eval_questions = ["What is the turing test?", 
                  "What is the rational agent approach?"]

eval_answers = ["The Turing test, originally called the imitation game by Alan Turing in 1950,[2] is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. Turing proposed that a human evaluator would judge natural language conversations between a human and a machine designed to generate human-like responses.",
                ""]

eval_answers = [[a] for a in eval_answers]

In [81]:
result = evaluate(query_engine, metrics, eval_questions, eval_answers) #Takes long to run. 5-10min depending on the number of questions and answers

evaluating with [faithfulness]


100%|██████████| 1/1 [03:24<00:00, 204.46s/it]


evaluating with [answer_relevancy]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


evaluating with [context_precision]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:08<00:00,  8.09s/it]


evaluating with [context_recall]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:37<00:00, 37.85s/it]


evaluating with [harmfulness]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:34<00:00, 34.74s/it]


In [83]:
result.to_pandas()

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,What is the turing test?,[6 Chapter 1.Introduction\ntheso-called totalT...,The Turing Test is a theoretical framework for...,"[The Turing test, originally called the imitat...",0.285714,0.920805,0.0,1.0,0
